In [ ]:
# Import libraries
import time
from tqdm import tqdm

import copy

import math

import matplotlib.pyplot as plt
import matplotlib.font_manager as font_manager
import matplotlib.gridspec as gridspec
import matplotlib.colors as col

from sklearn.metrics import roc_curve, auc
from sklearn.metrics import precision_recall_curve
from sklearn.metrics import average_precision_score
from sklearn.metrics import matthews_corrcoef
from sklearn.metrics import confusion_matrix
from sklearn.metrics import mean_squared_error
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score

import seaborn as sns

from sklearn.metrics import roc_curve, auc
from sklearn.metrics import precision_recall_curve
from sklearn.metrics import average_precision_score
from sklearn.metrics import matthews_corrcoef
from sklearn.metrics import confusion_matrix
from sklearn.metrics import mean_squared_error
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score

import numpy as np
np.random.seed(123)  
from numpy.ma import masked_array


import scipy as sp

import pandas as pd

import theano

from keras import backend as K
K.set_image_dim_ordering('th')

from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten, Reshape, Input
from keras.utils import np_utils
from keras.utils.np_utils import to_categorical
from keras.layers.convolutional import Convolution2D, MaxPooling2D
from keras.layers import LSTM
from keras.layers.wrappers import TimeDistributed
from keras.models import Model

import os
import gc

raster = 200
crime_type = "E05"
date_from = "2013-06-16"
date_to = "2017-03-08"
dist = 5
main_dir = "/home/mori/Documents/4_semestr/DIP"
main_dir = "/home/mori/DIP"

startcolor = '#fff2e5'
midcolor = '#f2a285'
endcolor = '#c40d21'    
cmap1 = col.LinearSegmentedColormap.from_list('own1',[startcolor,midcolor,endcolor])
plt.cm.register_cmap(name = 'wrong', cmap=cmap1)

startcolor = '#f3ffe5' 
midcolor = '#b4f185'   
endcolor = 'darkgreen'   
cmap2 = col.LinearSegmentedColormap.from_list('own2',[startcolor,midcolor,endcolor])
plt.cm.register_cmap(name = 'correct', cmap=cmap2)

classes=[0,1]

def find_max_mcc_threshold(true_values, predictions, thresholds):
    max_mcc = -1
    max_threshold = thresholds[0]
    max_predictions = predictions[:]
    stop = 0
    for i in range(1,thresholds.shape[0]):
        tmp_threshold = thresholds[i]
        tmp_predictions = copy.deepcopy(predictions)
        tmp_predictions[tmp_predictions >= tmp_threshold] = int(1)
        tmp_predictions[tmp_predictions < 1] = int(0)
        tmp_mcc = matthews_corrcoef(true_values, tmp_predictions)
        if tmp_mcc > max_mcc:
            max_mcc = tmp_mcc
            max_threshold = tmp_threshold
            max_predictions = copy.deepcopy(tmp_predictions)
            stop = 0
        else:
            stop += 1
        if stop == 5000:
            break;
    return max_mcc, max_threshold, max_predictions.astype(int)

    
def print_save_single_roc_threshold(true_values, predictions, title, fig_name):
    fpr, tpr, thresholds = roc_curve(true_values, predictions)
    prediction_auc = auc(fpr, tpr)
    
    mcc_max, threshold, predictions = find_max_mcc_threshold(true_values, predictions, thresholds)
    acc = accuracy_score(true_values, predictions)
    f1 = f1_score(true_values, predictions)
    th_index = thresholds.tolist().index(threshold)

    title_font = { 'size':'17', 'color':'black', 'weight':'normal', 'verticalalignment':'bottom'}
    axis_font = {'size':'15'}
    cb_font = {'size':'15', 'horizontalalignment':'left'}
    font_path = "/usr/share/fonts/truetype/msttcorefonts/Arial.ttf"
    font_prop = font_manager.FontProperties(size=15)
    
    
    fig = plt.figure(figsize=(15,7))
    gs1 = gridspec.GridSpec(1, 2)
    ax_list = [fig.add_subplot(ss) for ss in gs1]
    ax1 =  ax_list[0]
    ax2 =  ax_list[1]
    lw = 2

    ax1.scatter(fpr[th_index], tpr[th_index], s=75, c="red", label='Optimal threshold \n(cutoff = %0.4f)' % (threshold))
    ax1.plot(fpr, tpr, color="black", lw=lw, label='ROC curve (area = %0.4f)' % prediction_auc)
    ax1.plot([0, 1], [0, 1], color='gray', lw=lw, linestyle='--')
    ax1.set_xlim([0.0, 1.0])
    ax1.set_ylim([0.0, 1.05])
    ax1.set_xlabel('False Positive Rate', **axis_font)
    ax1.set_ylabel('True Positive Rate', **axis_font)
    ax1.set_title("Roc analysis",  **title_font)
    ax1.legend(loc="lower right", prop= font_prop)
    
    cm = confusion_matrix(true_values, predictions)
    cm_n = cm / cm.sum(axis=1)[:, np.newaxis]
    cm_f = [[1,0],[1,0]]
    
    mask1 = [[0,  1], [1, 0]]
    mask2 = [[1,  0], [0, 1]]
    cm1 = masked_array(cm_n,mask1)
    cm2 = masked_array(cm_n,mask2)
    cm1f = masked_array(cm_f,mask1)
    cm2f = masked_array(cm_f,mask2)
    
    cmap1=plt.cm.get_cmap("correct")
    cmap2=plt.cm.get_cmap("wrong")
    
    p2f = ax2.imshow(cm2f,interpolation='nearest',cmap=cmap2)
    p1f = ax2.imshow(cm1f,interpolation='nearest',cmap=cmap1)
   
    p2 = ax2.imshow(cm2,interpolation='nearest',cmap=cmap2)
    p1 = ax2.imshow(cm1,interpolation='nearest',cmap=cmap1)
    
    cb2 = plt.colorbar(p2,shrink=0.5)
    cb2.set_clim(0, 1)
    cb2.remove()
    
    cb1 = plt.colorbar(p1,shrink=0.5)
    cb1.set_clim(0, 1)
    cb1.remove()
    
    cb2 = plt.colorbar(p2f,shrink=0.5)
    cb2.set_clim(0, cm1.sum())
    cb2.ax.get_xaxis().labelpad = 10
    cb2.ax.set_xlabel('False', **cb_font)
    cb2.ax.tick_params(labelsize=15)

    cb1 = plt.colorbar(p1f,shrink=0.5)
    cb1.set_clim(0, cm1.sum())
    cb1.ax.get_xaxis().labelpad = 10
    cb1.ax.set_xlabel('True', **cb_font)
    cb1.ax.tick_params(labelsize=15)

    ax2.grid(False)

    ax2.set_title("Confusion matrix\n\nACC: %0.3f   F1: %0.3f   MCC: %0.3f" % (acc, f1, mcc_max), **title_font)

    tick_marks = np.arange(len(classes))
    ax2.set_xticks(tick_marks)
    ax2.set_yticks(tick_marks)
    ax2.set_xticklabels(classes)
    ax2.set_yticklabels(classes)
    
    thresh = 0.6

    ax2.text(0, 0, "%d\n(%0.2f)" % (cm[0,0], cm_n[0,0]), horizontalalignment="center", color="white" if cm_n[0, 0] > thresh else "black", fontproperties=font_prop)
    ax2.text(1, 1, "%d\n(%0.2f)" % (cm[1,1], cm_n[1,1]), horizontalalignment="center", color="white" if cm_n[1, 1] > thresh else "black", fontproperties=font_prop)
    ax2.text(1, 0, "%d\n(%0.2f)" % (cm[0,1], cm_n[0,1]), horizontalalignment="center", color="white" if cm_n[0, 1] > thresh else "black", fontproperties=font_prop)
    ax2.text(0, 1, "%d\n(%0.2f)" % (cm[1,0], cm_n[1,0]), horizontalalignment="center", color="white" if cm_n[1, 0] > thresh else "black", fontproperties=font_prop)

    ax2.set_ylabel('True label', **axis_font)
    ax2.set_xlabel('Predicted label', **axis_font)
    
    for label in (ax2.get_xticklabels() + ax2.get_yticklabels() + ax1.get_xticklabels() + ax1.get_yticklabels()):
        label.set_fontsize(13)
    
    plt.suptitle(title, fontsize=20)   
    
    plt.tight_layout(pad=1, w_pad=1, h_pad=1)
    plt.savefig("%s/output/python/keras/images/%s.png" % (main_dir, fig_name))
    plt.show()
    return "%0.5f,%0.5f,%0.5f,%0.5f,%0.5f,%d,%d,%d,%d,%0.5f,%0.5f,%0.5f,%0.5f" % (prediction_auc,acc,f1,mcc_max,threshold,cm[0,0],cm[1,1],cm[0,1],cm[1,0], cm_n[0,0],cm_n[1,1],cm_n[0,1],cm_n[1,0])
    

# Load data
dir_neigh = "%s/output/python/neighbour_arrays_%d_%s_%s_%s_x_%d_s_select.csv" % (main_dir, raster,crime_type, date_from, date_to, dist)
data_neigh = pd.read_csv(dir_neigh)

data_neigh = data_neigh[data_neigh.date > '2013-07-06']

data_neigh.sort_values("date", inplace=True)
print(data_neigh.date.min(), data_neigh.date.max())
data_neigh.head()    



# Split data

train_days_percent = 0.7
validation_days_percent = 0.2

ids_count = np.array((data_neigh.drop_duplicates("id", inplace = False)).iloc[:,0]).shape[0]
rows = data_neigh.shape[0]

total_days = rows/ids_count

train_rows = int(ids_count * total_days * train_days_percent)
validation_rows = int(ids_count * total_days * validation_days_percent)
test_rows = rows - train_rows - validation_rows
print("Train rows: %d, validation rows: %d, test rows: %d, all: %d < %d" % (train_rows, validation_rows, test_rows, (train_rows+validation_rows+test_rows), rows))

X_train = data_neigh.iloc[0:train_rows,:-5].values
X_validation = data_neigh.iloc[(train_rows):(train_rows+validation_rows),:-5].values
X_test = data_neigh.iloc[(train_rows+validation_rows):,:-5].values

y_train = data_neigh.iloc[0:train_rows, -1].values
y_validation = data_neigh.iloc[(train_rows):(train_rows+validation_rows),-1].values
y_test = data_neigh.iloc[(train_rows+validation_rows):,-1].values

n_train = y_train.shape[0]
n_validation = y_validation.shape[0]
n_test = y_test.shape[0]

print(n_train, n_validation, n_test, (n_train + n_validation + n_test), rows)
print((data_neigh.iloc[0:train_rows,:]).date.min(), (data_neigh.iloc[(train_rows):(train_rows+validation_rows),:]).date.min(), (data_neigh.iloc[(train_rows+validation_rows):,:]).date.min())
print(y_test[y_test > 0].sum())

max_y = int(data_neigh["crimecount"].max())

normalize_y = False

if normalize_y:
    y_train /= max_y
    y_validation /= max_y
    y_test /= max_y

binary = True

if binary:
    np.clip(y_train, 0, 1, out=y_train)
    np.clip(y_validation, 0, 1, out=y_validation)
    np.clip(y_test, 0, 1, out=y_test)
    n_classes = 2
    
    max_y = 1
    data_neigh.crimecount[data_neigh["crimecount"] > 0] = 1
    hist, bin_edges = np.histogram(data_neigh["crimecount"])
    plt.bar(bin_edges[:-1], hist, width = 1)
    plt.xlim(min(bin_edges), max(bin_edges))
    plt.show()
    print(hist)
else:
    n_classes = max_y + 1
    hist, bin_edges = np.histogram(data[y])
    plt.bar(bin_edges[:-1], hist, width = 1)
    plt.xlim(min(bin_edges), max(bin_edges))
    plt.show() 
    print(hist)

# Reshape, normalize and categorize data
image_shape = 2*dist + 1
X_train = X_train.reshape(n_train, 1, image_shape, image_shape).astype('float32')
X_validation = X_validation.reshape(n_validation, 1, image_shape, image_shape).astype('float32')
X_test = X_test.reshape(n_test, 1, image_shape, image_shape).astype('float32')


# How many examples to look at during each training iteration
batch_size = int(total_days)

# How many times to run through the full set of examples
epochs = 1

result_file_dir = "%s/output/python/model_results.csv" % main_dir
data_id = "neigh_data"

def train_test_model_save_results_cnn(model, model_id):
    model.fit(X_train,
          y_train,
          batch_size=batch_size,
          epochs=epochs,
          validation_data=(X_validation, y_validation))
    
    save_dir_model = "%s/output/python/keras/models/%s_%s.h5" % (main_dir, model_id, data_id)
    model.save(save_dir_model)

    prediction = model.predict(X_test)
    result_line = print_save_single_roc_threshold(y_test, prediction, "", "cnn_roc_%s_%s_th" %(model_id, data_id))
    params = str(model.to_json()) 
    with open(result_file_dir, 'a') as file:
        file.write("%s,%s,%s,%s\n" % (model_id, data_id, result_line, params))

# Model 1
model = Sequential()

# number of convolutional filters
n_filters = 8

# convolution filter size
n_conv = 2

# pooling window size
n_pool = 2

model.add(Convolution2D(
        n_filters, n_conv, n_conv,
        border_mode='valid',
        input_shape=(1, image_shape, image_shape)
))
model.add(Activation('relu'))

model.add(Convolution2D(n_filters, n_conv, n_conv))
model.add(Activation('relu'))

model.add(MaxPooling2D(pool_size=(n_pool, n_pool)))

model.add(Dropout(0.1))

model.add(Flatten())

model.add(Dense(100))
model.add(Activation('relu'))
model.add(Dropout(0.15))

model.add(Dense(1))
model.add(Activation('sigmoid'))

model.compile(
    loss='binary_crossentropy',
    optimizer='adam',
    metrics=['binary_accuracy']
)

train_test_model_save_results_cnn(model, "cnn_1")
print("1 Done.")

# Model 2
model = Sequential()

# number of convolutional filters
n_filters = 16

# convolution filter size
n_conv = 2

# pooling window size
n_pool = 2

model.add(Convolution2D(
        n_filters, n_conv, n_conv,
        border_mode='valid',
        input_shape=(1, image_shape, image_shape)
))
model.add(Activation('relu'))

model.add(Convolution2D(n_filters, n_conv, n_conv))
model.add(Activation('relu'))

model.add(MaxPooling2D(pool_size=(n_pool, n_pool)))

model.add(Dropout(0.1))

model.add(Flatten())

model.add(Dense(100))
model.add(Activation('relu'))
model.add(Dropout(0.15))

model.add(Dense(1))
model.add(Activation('sigmoid'))

model.compile(
    loss='binary_crossentropy',
    optimizer='adam',
    metrics=['binary_accuracy']
)

train_test_model_save_results_cnn(model, "cnn_2")
print("2 Done.")

# Model 3
model = Sequential()

# number of convolutional filters
n_filters = 32

# convolution filter size
n_conv = 2

# pooling window size
n_pool = 2

model.add(Convolution2D(
        n_filters, n_conv, n_conv,
        border_mode='valid',
        input_shape=(1, image_shape, image_shape)
))
model.add(Activation('relu'))

model.add(Convolution2D(n_filters, n_conv, n_conv))
model.add(Activation('relu'))

model.add(MaxPooling2D(pool_size=(n_pool, n_pool)))

model.add(Dropout(0.1))

model.add(Flatten())

model.add(Dense(100))
model.add(Activation('relu'))
model.add(Dropout(0.15))

model.add(Dense(1))
model.add(Activation('sigmoid'))

model.compile(
    loss='binary_crossentropy',
    optimizer='adam',
    metrics=['binary_accuracy']
)

train_test_model_save_results_cnn(model, "cnn_3")
print("3 Done.")

# Model 4
model = Sequential()

# number of convolutional filters
n_filters = 8

# convolution filter size
n_conv = 2

# pooling window size
n_pool = 2

model.add(Convolution2D(
        n_filters, n_conv, n_conv,
        border_mode='valid',
        input_shape=(1, image_shape, image_shape)
))
model.add(Activation('relu'))

model.add(Convolution2D(n_filters, n_conv, n_conv))
model.add(Activation('relu'))

model.add(Convolution2D(n_filters, n_conv, n_conv))
model.add(Activation('relu'))

model.add(Convolution2D(n_filters, n_conv, n_conv))
model.add(Activation('relu'))

model.add(MaxPooling2D(pool_size=(n_pool, n_pool)))

model.add(Dropout(0.1))

model.add(Flatten())

model.add(Dense(100))
model.add(Activation('relu'))
model.add(Dropout(0.15))

model.add(Dense(1))
model.add(Activation('sigmoid'))

model.compile(
    loss='binary_crossentropy',
    optimizer='adam',
    metrics=['binary_accuracy']
)

train_test_model_save_results_cnn(model, "cnn_4")
print("4 Done.")


# Model 5
model = Sequential()

# number of convolutional filters
n_filters = 16

# convolution filter size
n_conv = 2

# pooling window size
n_pool = 2

model.add(Convolution2D(
        n_filters, n_conv, n_conv,
        border_mode='valid',
        input_shape=(1, image_shape, image_shape)
))
model.add(Activation('relu'))

model.add(Convolution2D(n_filters, n_conv, n_conv))
model.add(Activation('relu'))

model.add(Convolution2D(n_filters, n_conv, n_conv))
model.add(Activation('relu'))

model.add(Convolution2D(n_filters, n_conv, n_conv))
model.add(Activation('relu'))

model.add(MaxPooling2D(pool_size=(n_pool, n_pool)))

model.add(Dropout(0.1))

model.add(Flatten())

model.add(Dense(100))
model.add(Activation('relu'))
model.add(Dropout(0.15))

model.add(Dense(1))
model.add(Activation('sigmoid'))

model.compile(
    loss='binary_crossentropy',
    optimizer='adam',
    metrics=['binary_accuracy']
)

train_test_model_save_results_cnn(model, "cnn_5")
print("5 Done.")


# Model 6
model = Sequential()

# number of convolutional filters
n_filters = 32

# convolution filter size
n_conv = 2

# pooling window size
n_pool = 2

model.add(Convolution2D(
        n_filters, n_conv, n_conv,
        border_mode='valid',
        input_shape=(1, image_shape, image_shape)
))
model.add(Activation('relu'))

model.add(Convolution2D(n_filters, n_conv, n_conv))
model.add(Activation('relu'))

model.add(Convolution2D(n_filters, n_conv, n_conv))
model.add(Activation('relu'))

model.add(Convolution2D(n_filters, n_conv, n_conv))
model.add(Activation('relu'))

model.add(MaxPooling2D(pool_size=(n_pool, n_pool)))

model.add(Dropout(0.1))

model.add(Flatten())

model.add(Dense(100))
model.add(Activation('relu'))
model.add(Dropout(0.15))

model.add(Dense(1))
model.add(Activation('sigmoid'))

model.compile(
    loss='binary_crossentropy',
    optimizer='adam',
    metrics=['binary_accuracy']
)

train_test_model_save_results_cnn(model, "cnn_6")
print("6 Done.")

# Model 7
model = Sequential()

# number of convolutional filters
n_filters = 64

# convolution filter size
n_conv = 2

# pooling window size
n_pool = 2

model.add(Convolution2D(
        n_filters, n_conv, n_conv,
        border_mode='valid',
        input_shape=(1, image_shape, image_shape)
))
model.add(Activation('relu'))

model.add(Convolution2D(n_filters, n_conv, n_conv))
model.add(Activation('relu'))

model.add(MaxPooling2D(pool_size=(n_pool, n_pool)))

model.add(Dropout(0.1))

model.add(Flatten())

model.add(Dense(100))
model.add(Activation('relu'))
model.add(Dropout(0.15))

model.add(Dense(1))
model.add(Activation('sigmoid'))

model.compile(
    loss='binary_crossentropy',
    optimizer='adam',
    metrics=['binary_accuracy']
)

train_test_model_save_results_cnn(model, "cnn_7")
print("7 Done.")

# Model 8
model = Sequential()

# number of convolutional filters
n_filters = 128

# convolution filter size
n_conv = 2

# pooling window size
n_pool = 2

model.add(Convolution2D(
        n_filters, n_conv, n_conv,
        border_mode='valid',
        input_shape=(1, image_shape, image_shape)
))
model.add(Activation('relu'))

model.add(Convolution2D(n_filters, n_conv, n_conv))
model.add(Activation('relu'))

model.add(MaxPooling2D(pool_size=(n_pool, n_pool)))

model.add(Dropout(0.1))

model.add(Flatten())

model.add(Dense(100))
model.add(Activation('relu'))
model.add(Dropout(0.15))

model.add(Dense(1))
model.add(Activation('sigmoid'))

model.compile(
    loss='binary_crossentropy',
    optimizer='adam',
    metrics=['binary_accuracy']
)

train_test_model_save_results_cnn(model, "cnn_8")
print("8 Done.")

# Model 9
model = Sequential()

# number of convolutional filters
n_filters = 256

# convolution filter size
n_conv = 2

# pooling window size
n_pool = 2

model.add(Convolution2D(
        n_filters, n_conv, n_conv,
        border_mode='valid',
        input_shape=(1, image_shape, image_shape)
))
model.add(Activation('relu'))

model.add(Convolution2D(n_filters, n_conv, n_conv))
model.add(Activation('relu'))

model.add(MaxPooling2D(pool_size=(n_pool, n_pool)))

model.add(Dropout(0.1))

model.add(Flatten())

model.add(Dense(100))
model.add(Activation('relu'))
model.add(Dropout(0.15))

model.add(Dense(1))
model.add(Activation('sigmoid'))

model.compile(
    loss='binary_crossentropy',
    optimizer='adam',
    metrics=['binary_accuracy']
)

train_test_model_save_results_cnn(model, "cnn_9")
print("9 Done.")

# Model 10
model = Sequential()

# number of convolutional filters
n_filters = 8

# convolution filter size
n_conv = 2

# pooling window size
n_pool = 2

model.add(Convolution2D(
        n_filters, n_conv, n_conv,
        border_mode='valid',
        input_shape=(1, image_shape, image_shape)
))
model.add(Activation('relu'))

model.add(Convolution2D(n_filters, n_conv, n_conv))
model.add(Activation('relu'))

model.add(Convolution2D(n_filters, n_conv, n_conv))
model.add(Activation('relu'))

model.add(Convolution2D(n_filters, n_conv, n_conv))
model.add(Activation('relu'))

model.add(Convolution2D(n_filters, n_conv, n_conv))
model.add(Activation('relu'))

model.add(Convolution2D(n_filters, n_conv, n_conv))
model.add(Activation('relu'))

model.add(Convolution2D(n_filters, n_conv, n_conv))
model.add(Activation('relu'))

model.add(Convolution2D(n_filters, n_conv, n_conv))
model.add(Activation('relu'))

model.add(MaxPooling2D(pool_size=(n_pool, n_pool)))

model.add(Dropout(0.1))

model.add(Flatten())

model.add(Dense(100))
model.add(Activation('relu'))
model.add(Dropout(0.15))

model.add(Dense(1))
model.add(Activation('sigmoid'))

model.compile(
    loss='binary_crossentropy',
    optimizer='adam',
    metrics=['binary_accuracy']
)

train_test_model_save_results_cnn(model, "cnn_10")
print("10 Done.")

# Model 11
model = Sequential()

# number of convolutional filters
n_filters = 16

# convolution filter size
n_conv = 2

# pooling window size
n_pool = 2

model.add(Convolution2D(
        n_filters, n_conv, n_conv,
        border_mode='valid',
        input_shape=(1, image_shape, image_shape)
))
model.add(Activation('relu'))

model.add(Convolution2D(n_filters, n_conv, n_conv))
model.add(Activation('relu'))

model.add(Convolution2D(n_filters, n_conv, n_conv))
model.add(Activation('relu'))

model.add(Convolution2D(n_filters, n_conv, n_conv))
model.add(Activation('relu'))

model.add(Convolution2D(n_filters, n_conv, n_conv))
model.add(Activation('relu'))

model.add(Convolution2D(n_filters, n_conv, n_conv))
model.add(Activation('relu'))

model.add(Convolution2D(n_filters, n_conv, n_conv))
model.add(Activation('relu'))

model.add(Convolution2D(n_filters, n_conv, n_conv))
model.add(Activation('relu'))

model.add(MaxPooling2D(pool_size=(n_pool, n_pool)))

model.add(Dropout(0.1))

model.add(Flatten())

model.add(Dense(100))
model.add(Activation('relu'))
model.add(Dropout(0.15))

model.add(Dense(1))
model.add(Activation('sigmoid'))

model.compile(
    loss='binary_crossentropy',
    optimizer='adam',
    metrics=['binary_accuracy']
)

train_test_model_save_results_cnn(model, "cnn_11")
print("11 Done.")

# Model 12
model = Sequential()

# number of convolutional filters
n_filters = 32

# convolution filter size
n_conv = 2

# pooling window size
n_pool = 2

model.add(Convolution2D(
        n_filters, n_conv, n_conv,
        border_mode='valid',
        input_shape=(1, image_shape, image_shape)
))
model.add(Activation('relu'))

model.add(Convolution2D(n_filters, n_conv, n_conv))
model.add(Activation('relu'))

model.add(Convolution2D(n_filters, n_conv, n_conv))
model.add(Activation('relu'))

model.add(Convolution2D(n_filters, n_conv, n_conv))
model.add(Activation('relu'))

model.add(Convolution2D(n_filters, n_conv, n_conv))
model.add(Activation('relu'))

model.add(Convolution2D(n_filters, n_conv, n_conv))
model.add(Activation('relu'))

model.add(Convolution2D(n_filters, n_conv, n_conv))
model.add(Activation('relu'))

model.add(Convolution2D(n_filters, n_conv, n_conv))
model.add(Activation('relu'))

model.add(MaxPooling2D(pool_size=(n_pool, n_pool)))

model.add(Dropout(0.1))

model.add(Flatten())

model.add(Dense(100))
model.add(Activation('relu'))
model.add(Dropout(0.15))

model.add(Dense(1))
model.add(Activation('sigmoid'))

model.compile(
    loss='binary_crossentropy',
    optimizer='adam',
    metrics=['binary_accuracy']
)

train_test_model_save_results_cnn(model, "cnn_12")
print("12 Done.")

In [ ]:
# Model 13
model = Sequential()

# number of convolutional filters
n_filters = 32

# convolution filter size
n_conv = 2

# pooling window size
n_pool = 2

model.add(Convolution2D(
        n_filters, n_conv, n_conv,
        border_mode='valid',
        input_shape=(1, image_shape, image_shape)
))
model.add(Activation('relu'))

model.add(Convolution2D(n_filters, n_conv, n_conv))
model.add(Activation('relu'))

model.add(Convolution2D(n_filters, n_conv, n_conv))
model.add(Activation('relu'))

model.add(Convolution2D(n_filters, n_conv, n_conv))
model.add(Activation('relu'))

model.add(MaxPooling2D(pool_size=(n_pool, n_pool)))

model.add(Dropout(0.1))

model.add(Flatten())

model.add(Dense(100))
model.add(Dense(100))
model.add(Dense(100))
model.add(Activation('relu'))
model.add(Dropout(0.15))

model.add(Dense(1))
model.add(Activation('sigmoid'))

model.compile(
    loss='binary_crossentropy',
    optimizer='adam',
    metrics=['binary_accuracy']
)

train_test_model_save_results_cnn(model, "cnn_13")
print("13 Done.")

In [ ]:
# Model 14
model = Sequential()

# number of convolutional filters
n_filters = 32

# convolution filter size
n_conv = 2

# pooling window size
n_pool = 2

model.add(Convolution2D(
        n_filters, n_conv, n_conv,
        border_mode='same',
        input_shape=(1, image_shape, image_shape)
))
model.add(Activation('relu'))

model.add(Convolution2D(n_filters, n_conv, n_conv))
model.add(Activation('relu'))

model.add(Convolution2D(n_filters, n_conv, n_conv))
model.add(Activation('relu'))

model.add(Convolution2D(n_filters, n_conv, n_conv))
model.add(Activation('relu'))

model.add(MaxPooling2D(pool_size=(n_pool, n_pool)))

model.add(Dropout(0.1))

model.add(Flatten())

model.add(Activation('relu'))
model.add(Dropout(0.15))

model.add(Dense(1))
model.add(Activation('sigmoid'))

model.compile(
    loss='binary_crossentropy',
    optimizer='adam',
    metrics=['binary_accuracy']
)

train_test_model_save_results_cnn(model, "cnn_14")
print("14 Done.")